In [1]:
from qm9 import QM9

%load_ext autoreload
%autoreload 2

In [45]:
data = QM9(r"../data/qm9/")

In [53]:
from rdkit import Chem

import pandas as pd
import numpy as np

path = "qm9_smiles.csv"

suppl = Chem.SmilesMolSupplier(path)

df = pd.read_csv(path)

df["target1"] = np.random.randn((len(df)))
df["target2"] = np.random.randn((len(df)))

df.set_index("smiles",inplace=True)

df.to_csv("qm9_target.csv")

## Testing SmilesDataset

In [34]:
from smiles_dataset import SmilesDataset

! rm -rf ../data/test/processed
data = SmilesDataset(root=r"../data/test", filename="qm9_target.csv", add_hydrogen=True)

Processing...
100%|██████████| 100/100 [00:00<00:00, 232.28it/s]
Done!


In [29]:
for graph in data:
    
    print(graph)

Data(x=[5, 5], edge_index=[2, 20], edge_attr=[20, 4], y=[1, 2], pos=[5, 3], z=[5], name='C', idx=[1])
Data(x=[4, 5], edge_index=[2, 12], edge_attr=[12, 4], y=[1, 2], pos=[4, 3], z=[4], name='N', idx=[1])
Data(x=[3, 5], edge_index=[2, 6], edge_attr=[6, 4], y=[1, 2], pos=[3, 3], z=[3], name='O', idx=[1])
Data(x=[4, 5], edge_index=[2, 12], edge_attr=[12, 4], y=[1, 2], pos=[4, 3], z=[4], name='C#C', idx=[1])
Data(x=[3, 5], edge_index=[2, 6], edge_attr=[6, 4], y=[1, 2], pos=[3, 3], z=[3], name='C#N', idx=[1])
Data(x=[4, 5], edge_index=[2, 12], edge_attr=[12, 4], y=[1, 2], pos=[4, 3], z=[4], name='C=O', idx=[1])
Data(x=[8, 5], edge_index=[2, 56], edge_attr=[56, 4], y=[1, 2], pos=[8, 3], z=[8], name='CC', idx=[1])
Data(x=[6, 5], edge_index=[2, 30], edge_attr=[30, 4], y=[1, 2], pos=[6, 3], z=[6], name='CO', idx=[1])
Data(x=[7, 5], edge_index=[2, 42], edge_attr=[42, 4], y=[1, 2], pos=[7, 3], z=[7], name='CC#C', idx=[1])
Data(x=[6, 5], edge_index=[2, 30], edge_attr=[30, 4], y=[1, 2], pos=[6, 3],